# Use Spark to predict credit risk with `ibm-watson-machine-learning`

This notebook introduces commands for model persistance to Watson Machine Learning repository, model deployment, and scoring.

Some familiarity with Python is helpful. This notebook uses Python 3.6 and Apache® Spark 2.4.

You will use **German Credit Risk** dataset.


## Learning goals

The learning goals of this notebook are:

-  Load a CSV file into an Apache® Spark DataFrame.
-  Explore data.
-  Prepare data for training and evaluation.
-  Persist a pipeline and model in Watson Machine Learning repository from tar.gz files.
-  Deploy a model for online scoring using Wastson Machine Learning API.
-  Score sample scoring data using the Watson Machine Learning API.
-  Explore and visualize prediction result using the plotly package.


## Contents

This notebook contains the following parts:

1.	[Set up](#setup)
2.	[Load and explore data](#load)
3.	[Persist model](#persistence)
4.	[Predict locally](#visualization)
5.	[Deploy and score in a Cloud](#scoring)
6.	[Clean up](#cleanup)
7.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-service-instance.html?context=analytics" target="_blank" rel="noopener no referrer">here</a>).

### Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud. You need to provide platform `api_key` and instance `location`.

You can use [IBM Cloud CLI](https://cloud.ibm.com/docs/cli/index.html) to retrieve platform API Key and instance location.

API Key can be generated in the following way:
```
ibmcloud login
ibmcloud iam api-key-create API_KEY_NAME
```

In result, get the value of `api_key` from the output.


Location of your WML instance can be retrieved in the following way:
```
ibmcloud login --apikey API_KEY -a https://cloud.ibm.com
ibmcloud resource service-instance WML_INSTANCE_NAME
```

In result, get the value of `location` from the output.

**Tip**: Your `Cloud API key` can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below. You can also get a service specific url by going to the [**Endpoint URLs** section of the Watson Machine Learning docs](https://cloud.ibm.com/apidocs/machine-learning).  You can check your instance location in your  <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance details.

You can also get service specific apikey by going to the [**Service IDs** section of the Cloud Console](https://cloud.ibm.com/iam/serviceids).  From that page, click **Create**, then copy the created key and paste it below.

**Action**: Enter your `api_key` and `location` in the following cell.

In [ ]:
api_key = 'PASTE YOUR PLATFORM API KEY HERE'
location = 'PASTE YOUR INSTANCE LOCATION HERE'

In [ ]:
wml_credentials = {
    "apikey": api_key,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}

### Install and import the `ibm-watson-machine-learning` package
**Note:** `ibm-watson-machine-learning` documentation can be found <a href="http://ibm-wml-api-pyclient.mybluemix.net/" target="_blank" rel="noopener no referrer">here</a>.

In [ ]:
!pip install -U ibm-watson-machine-learning

In [ ]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)

### Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have space already created, you can use [Deployment Spaces Dashboard](https://dataplatform.cloud.ibm.com/ml-runtime/spaces?context=cpdaas) to create one.

- Click New Deployment Space
- Create an empty space
- Select Cloud Object Storage
- Select Watson Machine Learning instance and press Create
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [ ]:
space_id = 'PASTE YOUR SPACE ID HERE'

You can use `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in Watson Machine Learning, you need to set **space** which you will be using.

In [ ]:
client.set.default_space(space_id)

### Test Spark

In [ ]:
try:
    from pyspark.sql import SparkSession
except:
    print('Error: Spark runtime is missing. If you are using Watson Studio change the notebook runtime to Spark.')
    raise

<a id="load"></a>
## 2. Load and explore data

In this section you will load the data as an Apache® Spark DataFrame and perform a basic exploration.
 

The csv file for German Credit Risk is available on the same repository as this notebook. Load the file to Apache® Spark DataFrame using code below.

In [ ]:
import os
from wget import download

sample_dir = 'spark_sample_model'
if not os.path.isdir(sample_dir):
    os.mkdir(sample_dir)
    
filename = os.path.join(sample_dir, 'credit_risk_training.csv')
if not os.path.isfile(filename):
    filename = download('https://github.com/IBM/watson-machine-learning-samples/raw/master/cloud/data/credit_risk/credit_risk_training.csv', out=sample_dir)

In [ ]:
spark = SparkSession.builder.getOrCreate()

df_data = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .load(filename)

Explore the loaded data by using the following Apache® Spark DataFrame methods:
-  print schema
-  print top ten records
-  count all records

In [ ]:
df_data.printSchema()

As you can see, the data contains 21 fields. Risk field is the one we would like to predict (label).

In [ ]:
df_data.show(n=5, truncate=False, vertical=True)

In [ ]:
print("Number of records: " + str(df_data.count()))

As you can see, the data set contains 5000 records.

### 2.1 Prepare data

In this subsection you will split your data into: train, test and predict datasets.

In [ ]:
splitted_data = df_data.randomSplit([0.8, 0.18, 0.02], 24)
train_data = splitted_data[0]
test_data = splitted_data[1]
predict_data = splitted_data[2]

print("Number of training records: " + str(train_data.count()))
print("Number of testing records : " + str(test_data.count()))
print("Number of prediction records : " + str(predict_data.count()))

As you can see our data has been successfully split into three datasets: 

-  The train data set, which is the largest group, is used for training.
-  The test data set will be used for model evaluation and is used to test the assumptions of the model.
-  The predict data set will be used for prediction.

<a id="persistence"></a>
## 3. Persist model

In this section you will learn how to store your pipeline and model in Watson Machine Learning repository by using python client libraries.

**Note**: Apache® Spark 2.4 is required.

#### Save training data in your Cloud Object Storage

ibm-cos-sdk library allows Python developers to manage Cloud Object Storage (COS).

In [ ]:
import ibm_boto3
from ibm_botocore.client import Config

**Action**: Put credentials from Object Storage Service in Bluemix here.

In [ ]:
cos_credentials = {
                  "apikey": "***",
                  "cos_hmac_keys": {
                    "access_key_id": "***",
                    "secret_access_key": "***"
                  },
                  "endpoints": "***",
                  "iam_apikey_description": "***",
                  "iam_apikey_name": "***",
                  "iam_role_crn": "***",
                  "iam_serviceid_crn": "***",
                  "resource_instance_id": "***"
                }

In [ ]:
connection_apikey = cos_credentials['apikey']
connection_resource_instance_id = cos_credentials["resource_instance_id"]
connection_access_key_id = cos_credentials['cos_hmac_keys']['access_key_id']
connection_secret_access_key = cos_credentials['cos_hmac_keys']['secret_access_key']

**Action**: Define the service endpoint we will use. <br>
**Tip**: You can find this information in Endpoints section of your Cloud Object Storage intance's dashbord.

In [ ]:
service_endpoint = 'https://s3.us.cloud-object-storage.appdomain.cloud'

You also need IBM Cloud authorization endpoint to be able to create COS resource object.

In [ ]:
auth_endpoint = 'https://iam.cloud.ibm.com/identity/token'

We create COS resource to be able to write data to Cloud Object Storage.

In [ ]:
cos = ibm_boto3.resource('s3',
                         ibm_api_key_id=cos_credentials['apikey'],
                         ibm_service_instance_id=cos_credentials['resource_instance_id'],
                         ibm_auth_endpoint=auth_endpoint,
                         config=Config(signature_version='oauth'),
                         endpoint_url=service_endpoint)

Now you will create bucket in COS and copy `training dataset` for model from **credit_risk_training.csv**.

In [ ]:
from uuid import uuid4

bucket_uid = str(uuid4())

score_filename = "credit_risk_training.csv"
buckets = ["credit-risk-" + bucket_uid]

In [ ]:
for bucket in buckets:
    if not cos.Bucket(bucket) in cos.buckets.all():
        print('Creating bucket "{}"...'.format(bucket))
        try:
            cos.create_bucket(Bucket=bucket)
        except ibm_boto3.exceptions.ibm_botocore.client.ClientError as e:
            print('Error: {}.'.format(e.response['Error']['Message']))

In [ ]:
bucket_obj = cos.Bucket(buckets[0])

print('Uploading data {}...'.format(score_filename))
with open(filename, 'rb') as f:
    bucket_obj.upload_fileobj(f, score_filename)
print('{} is uploaded.'.format(score_filename))

### 3.1: Save pipeline and model

In this subsection you will learn how to save pipeline and model artifacts to your Watson Machine Learning instance.

**Download pipeline and model archives**

In [ ]:
import os
from wget import download

sample_dir = 'spark_sample_model'
if not os.path.isdir(sample_dir):
    os.mkdir(sample_dir)
    
pipeline_filename = os.path.join(sample_dir, 'credit_risk_spark_pipeline.tar.gz')
if not os.path.isfile(pipeline_filename):
    pipeline_filename = download('https://github.com/IBM/watson-machine-learning-samples/raw/master/cloud/models/spark/credit-risk/model/credit_risk_spark_pipeline.tar.gz', out=sample_dir)
model_filename = os.path.join(sample_dir, 'credit_risk_spark_model.gz')
if not os.path.isfile(model_filename):
    model_filename = download('https://github.com/IBM/watson-machine-learning-samples/raw/master/cloud/models/spark/credit-risk/model/credit_risk_spark_model.gz', out=sample_dir)

**Store piepline and model**

To be able to store your Spark model, you need to provide a training data reference, this will allow to read the model schema automatically.

In [ ]:
training_data_references = [
                {
                    "type": "s3",
                    "connection": {
                        "access_key_id": connection_access_key_id,
                        "endpoint_url": service_endpoint,
                        "secret_access_key": connection_secret_access_key
                    },
                    "location": {
                        "bucket": buckets[0],
                        "path": score_filename,
                    },
                    "schema": {
                    "id": "training_schema",
                    "fields": [
                      {
                        "metadata": {},
                        "name": "CheckingStatus",
                        "nullable": True,
                        "type": "string"
                      },
                      {
                        "metadata": {},
                        "name": "LoanDuration",
                        "nullable": True,
                        "type": "integer"
                      },
                      {
                        "metadata": {},
                        "name": "CreditHistory",
                        "nullable": True,
                        "type": "string"
                      },
                      {
                        "metadata": {},
                        "name": "LoanPurpose",
                        "nullable": True,
                        "type": "string"
                      },
                      {
                        "metadata": {},
                        "name": "LoanAmount",
                        "nullable": True,
                        "type": "integer"
                      },
                      {
                        "metadata": {},
                        "name": "ExistingSavings",
                        "nullable": True,
                        "type": "string"
                      },
                      {
                        "metadata": {},
                        "name": "EmploymentDuration",
                        "nullable": True,
                        "type": "string"
                      },
                      {
                        "metadata": {},
                        "name": "InstallmentPercent",
                        "nullable": True,
                        "type": "integer"
                      },
                      {
                        "metadata": {},
                        "name": "Sex",
                        "nullable": True,
                        "type": "string"
                      },
                      {
                        "metadata": {},
                        "name": "OthersOnLoan",
                        "nullable": True,
                        "type": "string"
                      },
                      {
                        "metadata": {},
                        "name": "CurrentResidenceDuration",
                        "nullable": True,
                        "type": "integer"
                      },
                      {
                        "metadata": {},
                        "name": "OwnsProperty",
                        "nullable": True,
                        "type": "string"
                      },
                      {
                        "metadata": {},
                        "name": "Age",
                        "nullable": True,
                        "type": "integer"
                      },
                      {
                        "metadata": {},
                        "name": "InstallmentPlans",
                        "nullable": True,
                        "type": "string"
                      },
                      {
                        "metadata": {},
                        "name": "Housing",
                        "nullable": True,
                        "type": "string"
                      },
                      {
                        "metadata": {},
                        "name": "ExistingCreditsCount",
                        "nullable": True,
                        "type": "integer"
                      },
                      {
                        "metadata": {},
                        "name": "Job",
                        "nullable": True,
                        "type": "string"
                      },
                      {
                        "metadata": {},
                        "name": "Dependents",
                        "nullable": True,
                        "type": "integer"
                      },
                      {
                        "metadata": {},
                        "name": "Telephone",
                        "nullable": True,
                        "type": "string"
                      },
                      {
                        "metadata": {},
                        "name": "ForeignWorker",
                        "nullable": True,
                        "type": "string"
                      },
                      {
                        "metadata": {
                          "modeling_role": "target"
                        },
                        "name": "Risk",
                        "nullable": True,
                        "type": "string"
                      }
                    ]
                  }
                }
            ]

In [ ]:
published_model_details = client.repository.store_model(
    model=model_filename, 
    meta_props={
        client.repository.ModelMetaNames.NAME:'Credit Risk model',
        client.repository.ModelMetaNames.SPACE_UID: space_id,
        client.repository.ModelMetaNames.TYPE: "mllib_2.4",
        client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: client.software_specifications.get_id_by_name('spark-mllib_2.4'),
        client.repository.ModelMetaNames.TRAINING_DATA_REFERENCES: training_data_references,
        client.repository.ModelMetaNames.LABEL_FIELD: "Risk",
    }, 
    training_data=train_data, 
    pipeline=pipeline_filename)

In [ ]:
model_uid = client.repository.get_model_uid(published_model_details)
print(model_uid)

In [ ]:
client.repository.get_model_details(model_uid)

Get saved model metadata from Watson Machine Learning.

**Tip**: Use `client.repository.ModelMetaNames.show()` to get the list of available props.

In [ ]:
client.repository.ModelMetaNames.show()

### 3.2: Load model

In this subsection you will learn how to load back saved model from specified instance of Watson Machine Learning.

In [ ]:
loaded_model = client.repository.load(model_uid)

You can print for example model name to make sure that model has been loaded correctly.

In [ ]:
print(type(loaded_model))

<a id="visualization"></a>
## 4. Predict locally

In this section you will learn how to score test data using loaded model.

### 4.1: Make local prediction using previously loaded model and test data

In this subsection you will score *predict_data* data set.

In [ ]:
predictions = loaded_model.transform(predict_data)

Preview the results by calling the *show()* method on the predictions DataFrame.

In [ ]:
predictions.show(5)

By tabulating a count, you can see which product line is the most popular.

In [ ]:
predictions.select("predictedLabel").groupBy("predictedLabel").count().show(truncate=False)

<a id="scoring"></a>
## 5. Deploy and score in a Cloud

In this section you will learn how to create online scoring and to score a new data record using `ibm-watson-machine-learning`.

**Note:** You can also use REST API to deploy and score.
For more information about REST APIs, see the [Swagger Documentation](https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Deployments/deployments_create).

### 5.1: Create online scoring endpoint

Now you can create an online scoring endpoint. 

#### Create online deployment for published model

In [ ]:
deployment_details = client.deployments.create(
    model_uid, 
    meta_props={
        client.deployments.ConfigurationMetaNames.NAME: "Credit Risk model deployment",
        client.deployments.ConfigurationMetaNames.ONLINE: {}
    }
)

In [ ]:
deployment_details

Now, you can send new scoring records (new data) for which you would like to get predictions. To do that, execute the following sample code: 

In [ ]:
fields = ["CheckingStatus", "LoanDuration", "CreditHistory", "LoanPurpose", "LoanAmount", "ExistingSavings",
                  "EmploymentDuration", "InstallmentPercent", "Sex", "OthersOnLoan", "CurrentResidenceDuration",
                  "OwnsProperty", "Age", "InstallmentPlans", "Housing", "ExistingCreditsCount", "Job", "Dependents",
                  "Telephone", "ForeignWorker"]
values = [
    ["no_checking", 13, "credits_paid_to_date", "car_new", 1343, "100_to_500", "1_to_4", 2, "female", "none", 3,
     "savings_insurance", 46, "none", "own", 2, "skilled", 1, "none", "yes"],
    ["no_checking", 24, "prior_payments_delayed", "furniture", 4567, "500_to_1000", "1_to_4", 4, "male", "none",
     4, "savings_insurance", 36, "none", "free", 2, "management_self-employed", 1, "none", "yes"],
    ["0_to_200", 26, "all_credits_paid_back", "car_new", 863, "less_100", "less_1", 2, "female", "co-applicant",
     2, "real_estate", 38, "none", "own", 1, "skilled", 1, "none", "yes"],
    ["0_to_200", 14, "no_credits", "car_new", 2368, "less_100", "1_to_4", 3, "female", "none", 3, "real_estate",
     29, "none", "own", 1, "skilled", 1, "none", "yes"],
    ["0_to_200", 4, "no_credits", "car_new", 250, "less_100", "unemployed", 2, "female", "none", 3,
     "real_estate", 23, "none", "rent", 1, "management_self-employed", 1, "none", "yes"],
    ["no_checking", 17, "credits_paid_to_date", "car_new", 832, "100_to_500", "1_to_4", 2, "male", "none", 2,
     "real_estate", 42, "none", "own", 1, "skilled", 1, "none", "yes"],
    ["no_checking", 33, "outstanding_credit", "appliances", 5696, "unknown", "greater_7", 4, "male",
     "co-applicant", 4, "unknown", 54, "none", "free", 2, "skilled", 1, "yes", "yes"],
    ["0_to_200", 13, "prior_payments_delayed", "retraining", 1375, "100_to_500", "4_to_7", 3, "male", "none", 3,
     "real_estate", 37, "none", "own", 2, "management_self-employed", 1, "none", "yes"]
]

payload_scoring = {"input_data": [{"fields": fields, "values": values}]}
deployment_id = client.deployments.get_id(deployment_details)

client.deployments.score(deployment_id, payload_scoring)

<a id="cleanup"></a>
## 6. Clean up

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## 7. Summary and next steps     

 You successfully completed this notebook! You learned how to use Apache Spark machine learning as well as Watson Machine Learning for model creation and deployment. Check out our [Online Documentation](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-service-instance.html?context=analytics) for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Amadeusz Masny**, Python Software Developer in Watson Machine Learning at IBM

Copyright © 2020 IBM. This notebook and its source code are released under the terms of the MIT License.